In [6]:
import boto3
import pandas as pd
import io

# Setup S3 client
s3_client = boto3.client('s3')

# Download the file into memory
response = s3_client.get_object(Bucket='arjunp-cybersecurity-ml-data1', Key='raw-data/UNSW_NB15_training-set.csv')

# Read it into pandas
df = pd.read_csv(io.BytesIO(response['Body'].read()))

# Explore
print(df.shape)
print(df.columns)
print(df.head())
print(df['label'].value_counts())

(175341, 45)
Index(['id', 'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
       'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin',
       'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label'],
      dtype='object')
   id       dur proto service state  spkts  dpkts  sbytes  dbytes       rate  \
0   1  0.121478   tcp       -   FIN      6      4     258     172  74.087490   
1   2  0.649902   tcp       -   FIN     14     38     734   42014  78.473372   
2   3  1.623129   tcp       -   FIN      8     16     364   13186  14.170161   
3   4  1.681642   tcp     ftp   FIN     12     12     628     770  13.67

In [8]:
# --- 1. Drop irrelevant columns ---
df = df.drop(columns=['id', 'attack_cat'])

# --- 2. Feature engineering BEFORE encoding/scaling ---
df['byte_ratio'] = df['sbytes'] / (df['dbytes'] + 1)
df['is_common_port'] = df['ct_dst_sport_ltm'].isin([80, 443, 22]).astype(int)
df['flow_intensity'] = (df['spkts'] + df['dpkts']) / (df['dur'] + 1e-6)

# --- 3. One-hot encode categorical columns ---
categorical_cols = ['proto', 'service', 'state']
df = pd.get_dummies(df, columns=categorical_cols)

# Convert booleans to ints
df = df.astype({col: 'int' for col in df.columns if df[col].dtype == 'bool'})

# --- 4. Scale numerical features (except label) ---
from sklearn.preprocessing import StandardScaler

numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_cols.remove('label')

scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# --- Checks ---
print(df.shape)                                  # Final number of rows & columns
print(df.head())                                 # Preview first few rows
print(df.describe().T[['mean', 'std']])          # Confirm scaling stats
print(df[numerical_cols].mean().round(3))        # Should be ~0
print(df[numerical_cols].std().round(3))         # Should be ~1

(175341, 198)
        dur     spkts     dpkts    sbytes    dbytes      rate      sttl  \
0 -0.191029 -0.104456 -0.135769 -0.049134 -0.102726 -0.576371  0.703839   
1 -0.109485 -0.046014  0.172599 -0.046410  0.188544 -0.576345 -1.141901   
2  0.040699 -0.089845 -0.026933 -0.048527 -0.012133 -0.576734 -1.141901   
3  0.049729 -0.060624 -0.063212 -0.047016 -0.098563 -0.576737 -1.141901   
4 -0.140417 -0.075235 -0.117630 -0.047554 -0.102057 -0.576617  0.723268   

       dttl     sload     dload  ...  service_ssl  state_CON  state_ECO  \
0  1.578100 -0.389897 -0.273700  ...    -0.017874  -0.284764  -0.008273   
1  1.560002 -0.389928 -0.069233  ...    -0.017874  -0.284764  -0.008273   
2  1.560002 -0.389964 -0.252044  ...    -0.017874  -0.284764  -0.008273   
3  1.560002 -0.389958 -0.275821  ...    -0.017874  -0.284764  -0.008273   
4  1.560002 -0.389927 -0.275561  ...    -0.017874  -0.284764  -0.008273   

   state_FIN  state_INT  state_PAR  state_REQ  state_RST  state_URN  state_no  
0   

In [10]:
import sagemaker
from sagemaker import get_execution_role

# Create SageMaker session and define bucket
session = sagemaker.Session()
bucket = 'arjunp-cybersecurity-ml-data1'  # Replace with your actual S3 bucket name
processed_prefix = 'processed-data'      # Folder in S3 to store processed files

# Save preprocessed data locally
df.to_csv('preprocessed_data.csv', index=False)

# Upload to S3 inside the 'processed-data/' folder
s3_path = session.upload_data(
    path='preprocessed_data.csv',
    bucket=bucket,
    key_prefix=processed_prefix
)

print(f"Preprocessed data uploaded to: {s3_path}")

Preprocessed data uploaded to: s3://arjunp-cybersecurity-ml-data1/processed-data/preprocessed_data.csv
